In [ ]:
pip install botorch

In [2]:
import pandas as pd
import torch

train_data = pd.read_excel('data_sample_multiple_components_XYLOSE_9objs.xlsx')
#print(train_data)
train_x = train_data.iloc[:,[0,1]].values
#print(train_X)
train_x = torch.tensor(train_x, dtype=torch.float32)
print(train_x)

train_obj = train_data.iloc[:,2:].values
#print(train_Y)
train_obj = torch.tensor(train_obj, dtype=torch.float32)
print(train_obj)

tensor([[0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000]])
tensor([[ 0.1151,  0.0591,  0.0179,  0.1398,  0.0145,  0.1592,  0.0000,  0.0449,
         -0.4976],
        [ 0.1204,  0.0640,  0.0071,  0.0992,  0.0158,  0.1706,  0.0000,  0.0681,
         -0.5348],
        [ 0.1208,  0.0784,  0.0150,  0.1011,  0.0197,  0.2304,  0.0000,  0.0672,
         -0.5016],
        [ 0.1188,  0.0672,  0.0133,  0.1134,  0.0167,  0.1867,  0.0000,  0.

In [4]:
from botorch import fit_gpytorch_model
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.transforms.outcome import Standardize
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.optim.optimize import optimize_acqf

torch.manual_seed(0)

NOISE_SE = torch.tensor([1,1,1,1,1,1,1,1,1])
#print(NOISE_SE)

# define models for objective and constraint
models = []
for i in range(train_obj.shape[-1]):
    train_y = train_obj[..., i:i+1]
    train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
    models.append(FixedNoiseGP(train_x, train_y, train_yvar, outcome_transform=Standardize(m=1)))
model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_model(mll)

BATCH_SIZE = 10
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[4.1770e-02, 1.5790e-01],
        [1.3619e-02, 2.5995e+00],
        [1.9970e-02, 2.2111e-01],
        [4.4772e-02, 2.0989e+00],
        [3.9377e-02, 1.7199e+00],
        [1.2088e-03, 3.8620e-01],
        [4.0980e-03, 6.6677e-01],
        [5.1903e-03, 2.7432e+00],
        [3.3898e-02, 2.4039e+00],
        [2.4758e-02, 2.5757e-01]])


In [5]:
for i in range(9):
    print(model.models[i].covar_module.outputscale, 
          model.models[i].covar_module.base_kernel.lengthscale, 
          model.models[i].likelihood.noise)

tensor(6.2247, grad_fn=<SoftplusBackward0>) tensor([[0.3333, 0.3334]], grad_fn=<SoftplusBackward0>) tensor([1063.9309, 1063.9309, 1063.9309, 1063.9309, 1063.9309, 1063.9309,
        1063.9309, 1063.9309, 1063.9309, 1063.9309, 1063.9309, 1063.9309,
        1063.9309, 1063.9309, 1063.9309, 1063.9309, 1063.9309, 1063.9309,
        1063.9309, 1063.9309, 1063.9309, 1063.9309, 1063.9309, 1063.9309])
tensor(6.5035, grad_fn=<SoftplusBackward0>) tensor([[0.3331, 0.3340]], grad_fn=<SoftplusBackward0>) tensor([2877.7764, 2877.7764, 2877.7764, 2877.7764, 2877.7764, 2877.7764,
        2877.7764, 2877.7764, 2877.7764, 2877.7764, 2877.7764, 2877.7764,
        2877.7764, 2877.7764, 2877.7764, 2877.7764, 2877.7764, 2877.7764,
        2877.7764, 2877.7764, 2877.7764, 2877.7764, 2877.7764, 2877.7764])
tensor(6.6537, grad_fn=<SoftplusBackward0>) tensor([[0.3333, 0.3333]], grad_fn=<SoftplusBackward0>) tensor([103948.1562, 103948.1562, 103948.1562, 103948.1562, 103948.1562,
        103948.1562, 103948.1562,